In [1]:
// Import necessary dependencies
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.13.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.17.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-serialization-json:1.5.1")



In [2]:
import com.fasterxml.jackson.annotation.JsonFormat
import com.fasterxml.jackson.module.kotlin.jacksonObjectMapper
import com.fasterxml.jackson.module.kotlin.readValue
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.variable.PlanningListVariable
import ai.timefold.solver.core.api.score.buildin.hardsoftlong.HardSoftLongScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.stream.ConstraintCollectors
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.Solver
import java.io.File
import java.time.Duration
import java.time.LocalTime


In [3]:
data class Location(
    val latitude: Double,
    val longitude: Double
) {
    fun distanceTo(other: Location): Double {
        val earthRadius = 6371e3 // Earth radius in meters
        val lat1 = Math.toRadians(latitude)
        val lat2 = Math.toRadians(other.latitude)
        val deltaLat = Math.toRadians(other.latitude - latitude)
        val deltaLon = Math.toRadians(other.longitude - longitude)

        val a = Math.sin(deltaLat / 2) * Math.sin(deltaLat / 2) +
                Math.cos(lat1) * Math.cos(lat2) *
                Math.sin(deltaLon / 2) * Math.sin(deltaLon / 2)
        val c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1 - a))

        val distance = earthRadius * c // Distance in meters
        return distance
    }
}


In [5]:
// Cell 4: Define Truck and Client Classes together
import com.fasterxml.jackson.annotation.JsonProperty
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.variable.PlanningVariable
import ai.timefold.solver.core.api.domain.variable.PlanningListVariable
import ai.timefold.solver.core.api.domain.variable.InverseRelationShadowVariable
import ai.timefold.solver.core.api.domain.variable.NextElementShadowVariable
import ai.timefold.solver.core.api.domain.variable.PreviousElementShadowVariable
import ai.timefold.solver.core.api.domain.lookup.PlanningId
import java.time.LocalTime
import java.time.format.DateTimeFormatter

// Define Client class
@PlanningEntity
data class Client(
    @JsonProperty("Client_ID")
    val clientId: String,

    @JsonProperty("Need")
    val need: String, // "Waste", "Enrobé", "Grue"

    @JsonProperty("Volume_t")
    val volumeT: Double,

    @JsonProperty("Volume_kg")
    val volumeKg: Double,

    @JsonProperty("Access_Difficulty")
    val accessDifficulty: String, // "Yes" or "No"

    @JsonProperty("Time_Window_Label")
    val timeWindowLabel: String, // "Morning", "Afternoon", "All Day"

    @JsonProperty("Time_Window_Start")
    val timeWindowStart: String, // "08:00", "13:00", etc.

    @JsonProperty("Time_Window_End")
    val timeWindowEnd: String,

    @JsonProperty("Latitude")
    val latitude: Double,

    @JsonProperty("Longitude")
    val longitude: Double
) {
    val location: Location = Location(latitude, longitude)

    // Convert "Yes"/"No" to Boolean
    val hasAccessDifficulty: Boolean
        get() = accessDifficulty.equals("Yes", ignoreCase = true)

    // Time windows as LocalTime
    val timeWindowStartTime: LocalTime
        get() = LocalTime.parse(timeWindowStart, DateTimeFormatter.ofPattern("HH:mm"))

    val timeWindowEndTime: LocalTime
        get() = LocalTime.parse(timeWindowEnd, DateTimeFormatter.ofPattern("HH:mm"))

    // Planning variables

    @InverseRelationShadowVariable(sourceVariableName = "assignedClients")
    var truck: Truck? = null

    @PreviousElementShadowVariable(sourceVariableName = "assignedClients")
    var previousClient: Client? = null

    @NextElementShadowVariable(sourceVariableName = "assignedClients")
    var nextClient: Client? = null

    // No-arg constructor for Timefold
    constructor() : this(
        clientId = "",
        need = "",
        volumeT = 0.0,
        volumeKg = 0.0,
        accessDifficulty = "No",
        timeWindowLabel = "",
        timeWindowStart = "00:00",
        timeWindowEnd = "00:00",
        latitude = 0.0,
        longitude = 0.0
    )

    override fun toString(): String = clientId
}

// Define Truck class
@PlanningEntity
data class Truck(
    @PlanningId
    val truckId: String,

    val truckType: String, // e.g., "Multilift 4 essieux", "Multilift 4 essieux Tridem", "Camion-grue 4 essieux"

    val capacityKg: Double,

    val equipment: String, // e.g., "Benne 10 m3", "Benne 20 m3", "Pont 13 m3", "Thermosilo"

    val canAccessDifficultLocations: Boolean,

    val homeLocation: Location // Warehouse location
) {
    @PlanningListVariable(valueRangeProviderRefs = ["clientRange"])
    var assignedClients: MutableList<Client> = mutableListOf()

    // No-arg constructor for Timefold
    constructor() : this(
        truckId = "",
        truckType = "",
        capacityKg = 0.0,
        equipment = "",
        canAccessDifficultLocations = false,
        homeLocation = Location(0.0, 0.0)
    )

    override fun toString(): String = truckId
}


In [6]:
val clientsJson = """


[
    {
        "Client_ID":"C001",
        "Need":"Grue",
        "Volume_t":13.73,
        "Volume_kg":13726.61,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8727114265,
        "Longitude":2.4152913367
    },
    {
        "Client_ID":"C002",
        "Need":"Enrobée",
        "Volume_t":16.51,
        "Volume_kg":16511.92,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8337749631,
        "Longitude":2.4624881962
    },
    {
        "Client_ID":"C003",
        "Need":"Grue",
        "Volume_t":14.97,
        "Volume_kg":14966.65,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8995203229,
        "Longitude":2.3830675345
    },
    {
        "Client_ID":"C004",
        "Need":"Grue",
        "Volume_t":16.41,
        "Volume_kg":16406.54,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8552772065,
        "Longitude":2.3719840732
    },
    {
        "Client_ID":"C005",
        "Need":"Waste",
        "Volume_t":10.42,
        "Volume_kg":10417.48,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8267392211,
        "Longitude":2.3840461925
    },
    {
        "Client_ID":"C006",
        "Need":"Enrobée",
        "Volume_t":17.52,
        "Volume_kg":17524.55,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8798084826,
        "Longitude":2.2785357718
    },
    {
        "Client_ID":"C007",
        "Need":"Enrobée",
        "Volume_t":17.49,
        "Volume_kg":17491.73,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8424366831,
        "Longitude":2.3312467777
    },
    {
        "Client_ID":"C008",
        "Need":"Enrobée",
        "Volume_t":13.61,
        "Volume_kg":13605.94,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8297649991,
        "Longitude":2.3169421334
    },
    {
        "Client_ID":"C009",
        "Need":"Enrobée",
        "Volume_t":14.16,
        "Volume_kg":14164.58,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8632249724,
        "Longitude":2.3568582611
    },
    {
        "Client_ID":"C010",
        "Need":"Waste",
        "Volume_t":12.89,
        "Volume_kg":12893.21,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8357622912,
        "Longitude":2.4148355918
    },
    {
        "Client_ID":"C011",
        "Need":"Waste",
        "Volume_t":13.35,
        "Volume_kg":13348.32,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.885563268,
        "Longitude":2.4053477927
    },
    {
        "Client_ID":"C012",
        "Need":"Waste",
        "Volume_t":10.18,
        "Volume_kg":10182.27,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8344111745,
        "Longitude":2.3409284004
    },
    {
        "Client_ID":"C013",
        "Need":"Waste",
        "Volume_t":18.69,
        "Volume_kg":18693.3,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8444315354,
        "Longitude":2.2819629252
    },
    {
        "Client_ID":"C014",
        "Need":"Waste",
        "Volume_t":14.21,
        "Volume_kg":14212.6,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8882465175,
        "Longitude":2.264484004
    },
    {
        "Client_ID":"C015",
        "Need":"Waste",
        "Volume_t":12.43,
        "Volume_kg":12430.33,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8567207885,
        "Longitude":2.2987460852
    },
    {
        "Client_ID":"C016",
        "Need":"Enrobée",
        "Volume_t":15.98,
        "Volume_kg":15981.13,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8360376647,
        "Longitude":2.3018284564
    },
    {
        "Client_ID":"C017",
        "Need":"Waste",
        "Volume_t":18.58,
        "Volume_kg":18577.18,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8563111256,
        "Longitude":2.4515215805
    },
    {
        "Client_ID":"C018",
        "Need":"Waste",
        "Volume_t":13.54,
        "Volume_kg":13536.02,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8843489829,
        "Longitude":2.3179995845
    },
    {
        "Client_ID":"C019",
        "Need":"Waste",
        "Volume_t":14.38,
        "Volume_kg":14380.32,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8277019945,
        "Longitude":2.3644008554
    },
    {
        "Client_ID":"C020",
        "Need":"Waste",
        "Volume_t":19.88,
        "Volume_kg":19877.06,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.852508881,
        "Longitude":2.4209095852
    },
    {
        "Client_ID":"C021",
        "Need":"Waste",
        "Volume_t":15.71,
        "Volume_kg":15708.49,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8958473007,
        "Longitude":2.2283797347
    },
    {
        "Client_ID":"C022",
        "Need":"Waste",
        "Volume_t":17.75,
        "Volume_kg":17747.05,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8840654658,
        "Longitude":2.3356365874
    },
    {
        "Client_ID":"C023",
        "Need":"Waste",
        "Volume_t":11.92,
        "Volume_kg":11916.03,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8998471212,
        "Longitude":2.322535139
    },
    {
        "Client_ID":"C024",
        "Need":"Waste",
        "Volume_t":12.86,
        "Volume_kg":12862.85,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8930421522,
        "Longitude":2.3358789851
    },
    {
        "Client_ID":"C025",
        "Need":"Enrobée",
        "Volume_t":17.04,
        "Volume_kg":17044.63,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8509098728,
        "Longitude":2.2290630239
    },
    {
        "Client_ID":"C026",
        "Need":"Waste",
        "Volume_t":13.99,
        "Volume_kg":13990.61,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8945021227,
        "Longitude":2.2973676485
    },
    {
        "Client_ID":"C027",
        "Need":"Waste",
        "Volume_t":14.33,
        "Volume_kg":14333.43,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8570057014,
        "Longitude":2.2757384372
    },
    {
        "Client_ID":"C028",
        "Need":"Waste",
        "Volume_t":10.44,
        "Volume_kg":10438.7,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8295147289,
        "Longitude":2.3976309301
    },
    {
        "Client_ID":"C029",
        "Need":"Enrobée",
        "Volume_t":15.05,
        "Volume_kg":15053.1,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8313453015,
        "Longitude":2.2891161603
    },
    {
        "Client_ID":"C030",
        "Need":"Waste",
        "Volume_t":17.0,
        "Volume_kg":17002.53,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8546398289,
        "Longitude":2.3367049267
    },
    {
        "Client_ID":"C031",
        "Need":"Waste",
        "Volume_t":19.75,
        "Volume_kg":19748.83,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8764464984,
        "Longitude":2.387878084
    },
    {
        "Client_ID":"C032",
        "Need":"Waste",
        "Volume_t":14.83,
        "Volume_kg":14832.12,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.888965554,
        "Longitude":2.2507665485
    },
    {
        "Client_ID":"C033",
        "Need":"Enrobée",
        "Volume_t":13.98,
        "Volume_kg":13976.26,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8634868191,
        "Longitude":2.463648015
    },
    {
        "Client_ID":"C034",
        "Need":"Waste",
        "Volume_t":15.17,
        "Volume_kg":15174.54,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8325553338,
        "Longitude":2.2648644966
    },
    {
        "Client_ID":"C035",
        "Need":"Waste",
        "Volume_t":11.15,
        "Volume_kg":11150.33,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8491311632,
        "Longitude":2.4134520825
    },
    {
        "Client_ID":"C036",
        "Need":"Enrobée",
        "Volume_t":16.84,
        "Volume_kg":16836.07,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8224189719,
        "Longitude":2.2367828582
    },
    {
        "Client_ID":"C037",
        "Need":"Waste",
        "Volume_t":14.0,
        "Volume_kg":13998.22,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.85201415,
        "Longitude":2.342531956
    },
    {
        "Client_ID":"C038",
        "Need":"Enrobée",
        "Volume_t":16.83,
        "Volume_kg":16825.14,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8511520307,
        "Longitude":2.4396337885
    },
    {
        "Client_ID":"C039",
        "Need":"Grue",
        "Volume_t":15.75,
        "Volume_kg":15745.92,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8288579155,
        "Longitude":2.3021950667
    },
    {
        "Client_ID":"C040",
        "Need":"Waste",
        "Volume_t":13.89,
        "Volume_kg":13892.92,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8372274366,
        "Longitude":2.4186360279
    },
    {
        "Client_ID":"C041",
        "Need":"Grue",
        "Volume_t":10.22,
        "Volume_kg":10215.16,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8986725908,
        "Longitude":2.2921971364
    },
    {
        "Client_ID":"C042",
        "Need":"Grue",
        "Volume_t":10.03,
        "Volume_kg":10031.65,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8598342054,
        "Longitude":2.3115851055
    },
    {
        "Client_ID":"C043",
        "Need":"Waste",
        "Volume_t":19.93,
        "Volume_kg":19929.27,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8210212275,
        "Longitude":2.2962086898
    },
    {
        "Client_ID":"C044",
        "Need":"Waste",
        "Volume_t":11.96,
        "Volume_kg":11960.49,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8762368303,
        "Longitude":2.2660315935
    },
    {
        "Client_ID":"C045",
        "Need":"Enrobée",
        "Volume_t":15.01,
        "Volume_kg":15014.12,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8872887819,
        "Longitude":2.4086223577
    },
    {
        "Client_ID":"C046",
        "Need":"Enrobée",
        "Volume_t":13.72,
        "Volume_kg":13724.74,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8674790585,
        "Longitude":2.313550389
    },
    {
        "Client_ID":"C047",
        "Need":"Waste",
        "Volume_t":19.88,
        "Volume_kg":19877.12,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8753450245,
        "Longitude":2.2514940748
    },
    {
        "Client_ID":"C048",
        "Need":"Waste",
        "Volume_t":17.63,
        "Volume_kg":17626.71,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8295176506,
        "Longitude":2.2938424521
    },
    {
        "Client_ID":"C049",
        "Need":"Enrobée",
        "Volume_t":17.72,
        "Volume_kg":17715.78,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8431161596,
        "Longitude":2.3191251583
    },
    {
        "Client_ID":"C050",
        "Need":"Waste",
        "Volume_t":18.67,
        "Volume_kg":18668.69,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8664786498,
        "Longitude":2.2519575393
    }
]

""".trimIndent()


In [7]:
import com.fasterxml.jackson.module.kotlin.jacksonObjectMapper
import com.fasterxml.jackson.module.kotlin.readValue

val mapper = jacksonObjectMapper()

val clients: List<Client> = mapper.readValue(clientsJson)

println("Loaded ${clients.size} clients:")
clients.forEach { println(it) }


Loaded 50 clients:
C001
C002
C003
C004
C005
C006
C007
C008
C009
C010
C011
C012
C013
C014
C015
C016
C017
C018
C019
C020
C021
C022
C023
C024
C025
C026
C027
C028
C029
C030
C031
C032
C033
C034
C035
C036
C037
C038
C039
C040
C041
C042
C043
C044
C045
C046
C047
C048
C049
C050


In [8]:
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.score.buildin.hardsoftlong.HardSoftLongScore

@PlanningSolution
data class Schedule(
    @PlanningEntityCollectionProperty
    val trucks: List<Truck>,

    @ProblemFactCollectionProperty
    @ValueRangeProvider(id = "clientRange")
    val clients: List<Client>
) {
    @PlanningScore
    var score: HardSoftLongScore? = null

    // No-arg constructor for Timefold
    constructor() : this(
        trucks = emptyList(),
        clients = emptyList()
    )
}


In [9]:
import ai.timefold.solver.core.api.score.buildin.hardsoftlong.HardSoftLongScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider

class TruckRoutingConstraintProvider : ConstraintProvider {
    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint> {
        return arrayOf(
            capacityConstraint(constraintFactory),
            accessDifficultyConstraint(constraintFactory),
            equipmentConstraint(constraintFactory),
            minimizeTotalDistance(constraintFactory)
        )
    }

    // Constraint: Truck capacity must not be exceeded
    fun capacityConstraint(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Truck::class.java)
            .filter { truck ->
                val totalVolumeKg = truck.assignedClients.sumOf { it.volumeKg }
                totalVolumeKg > truck.capacityKg
            }
            .penalizeLong(HardSoftLongScore.ONE_HARD) { truck ->
                val totalVolumeKg = truck.assignedClients.sumOf { it.volumeKg }
                (totalVolumeKg - truck.capacityKg).toLong()
            }
            .asConstraint("CapacityConstraint")
    }

    // Constraint: Trucks that cannot access difficult locations should not be assigned clients with access difficulty
    fun accessDifficultyConstraint(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Client::class.java)
            .filter { client ->
                client.hasAccessDifficulty && client.truck != null && !client.truck!!.canAccessDifficultLocations
            }
            .penalize(HardSoftLongScore.ONE_HARD)
            .asConstraint("AccessDifficultyConstraint")
    }

    // Constraint: Trucks must have the appropriate equipment to satisfy client's need
    fun equipmentConstraint(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Client::class.java)
            .filter { client ->
                val truck = client.truck
                truck != null && !truckCanSatisfyClient(truck, client)
            }
            .penalize(HardSoftLongScore.ONE_HARD)
            .asConstraint("EquipmentConstraint")
    }

    // Soft Constraint: Minimize total distance travelled
    fun minimizeTotalDistance(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Truck::class.java)
            .penalizeLong(HardSoftLongScore.ONE_SOFT) { truck ->
                var totalDistance = 0.0
                var previousLocation = truck.homeLocation
                for (client in truck.assignedClients) {
                    totalDistance += previousLocation.distanceTo(client.location)
                    previousLocation = client.location
                }
                totalDistance += previousLocation.distanceTo(truck.homeLocation) // Return to warehouse
                totalDistance.toLong()
            }
            .asConstraint("MinimizeTotalDistance")
    }

    // Helper function to check if a truck can satisfy a client's need based on equipment
    fun truckCanSatisfyClient(truck: Truck, client: Client): Boolean {
        return when (client.need) {
            "Waste" -> truck.equipment in listOf("Benne 10 m3", "Benne 20 m3", "Pont 13 m3")
            "Enrobé" -> truck.equipment == "Thermosilo"
            "Grue" -> truck.truckType == "Camion-grue 4 essieux"
            else -> false
        }
    }
}


In [10]:
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.Solver
import ai.timefold.solver.core.config.solver.SolverConfig
import java.time.Duration

val solverFactory: SolverFactory<Schedule> = SolverFactory.create(SolverConfig()
    .withSolutionClass(Schedule::class.java)
    .withEntityClasses(Truck::class.java)
    .withConstraintProviderClass(TruckRoutingConstraintProvider::class.java)
    .withTerminationSpentLimit(Duration.ofSeconds(30))
)

println("Solving the problem ...")
val solver: Solver<Schedule> = solverFactory.buildSolver()

// Define warehouse location (replace with actual coordinates)
val warehouseLocation = Location(latitude = 48.8566, longitude = 2.3522) // Example: Paris coordinates

// Define trucks
val trucks = listOf(
    Truck(
        truckId = "T1",
        truckType = "Multilift 4 essieux",
        capacityKg = 20000.0,
        equipment = "Benne 10 m3",
        canAccessDifficultLocations = false,
        homeLocation = warehouseLocation
    ),
    Truck(
        truckId = "T2",
        truckType = "Multilift 4 essieux",
        capacityKg = 20000.0,
        equipment = "Benne 20 m3",
        canAccessDifficultLocations = false,
        homeLocation = warehouseLocation
    ),
    Truck(
        truckId = "T3",
        truckType = "Multilift 4 essieux",
        capacityKg = 20000.0,
        equipment = "Pont 13 m3",
        canAccessDifficultLocations = false,
        homeLocation = warehouseLocation
    ),
    Truck(
        truckId = "T4",
        truckType = "Multilift 4 essieux Tridem",
        capacityKg = 20000.0,
        equipment = "Benne 10 m3",
        canAccessDifficultLocations = true,
        homeLocation = warehouseLocation
    ),
    Truck(
        truckId = "T5",
        truckType = "Camion-grue 4 essieux",
        capacityKg = 8500.0,
        equipment = "",
        canAccessDifficultLocations = false,
        homeLocation = warehouseLocation
    )
)

// Create initial Schedule
val schedule = Schedule(
    trucks = trucks,
    clients = clients
)

// Solve the problem
val solution: Schedule = solver.solve(schedule)
println("Solving finished with score (${solution.score}).")

// Output the solution
solution.trucks.forEach { truck ->
    println("Truck ${truck.truckId} (${truck.truckType}, Equipment: ${truck.equipment}):")
    truck.assignedClients.forEach { client ->
        println(" - Client ${client.clientId} (${client.need}) at (${client.latitude}, ${client.longitude})")
    }
    println()
}


Solving the problem ...
Solving finished with score (-663465hard/-92747soft).
Truck T1 (Multilift 4 essieux, Equipment: Benne 10 m3):
 - Client C049 (Enrobée) at (48.8431161596, 2.3191251583)
 - Client C016 (Enrobée) at (48.8360376647, 2.3018284564)
 - Client C029 (Enrobée) at (48.8313453015, 2.2891161603)
 - Client C048 (Waste) at (48.8295176506, 2.2938424521)
 - Client C043 (Waste) at (48.8210212275, 2.2962086898)
 - Client C039 (Grue) at (48.8288579155, 2.3021950667)
 - Client C008 (Enrobée) at (48.8297649991, 2.3169421334)

Truck T2 (Multilift 4 essieux, Equipment: Benne 20 m3):
 - Client C046 (Enrobée) at (48.8674790585, 2.313550389)
 - Client C042 (Grue) at (48.8598342054, 2.3115851055)
 - Client C015 (Waste) at (48.8567207885, 2.2987460852)
 - Client C027 (Waste) at (48.8570057014, 2.2757384372)
 - Client C013 (Waste) at (48.8444315354, 2.2819629252)
 - Client C034 (Waste) at (48.8325553338, 2.2648644966)
 - Client C036 (Enrobée) at (48.8224189719, 2.2367828582)
 - Client C025 (